In [1]:
from sklearn import neighbors

In [2]:
knn=neighbors.KNeighborsClassifier(n_neighbors=2)

In [3]:
knn

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [4]:
knn.fit([[1],[2],[3],[4],[5],[6]], [0,0,0,1,1,1])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [5]:
knn.predict(1.5)

array([0])

In [6]:
knn.predict(3)

array([0])

In [7]:
knn.predict(37)

array([1])

In [8]:
knn.predict_proba(37)

array([[ 0.,  1.]])

In [9]:
knn.predict_proba(3)

array([[ 1.,  0.]])

In [10]:
knn.predict_proba(3.5)

array([[ 0.5,  0.5]])

In [11]:
import re

In [12]:
code_match=re.compile('<pre>(.*?)</pre>',re.MULTILINE|re.DOTALL)

In [13]:
link_match=re.compile('a href="http://.*?".*?>(.*?)</a>',re.MULTILINE|re.DOTALL)

In [14]:
def extract_features_from_body(s):
    link_count_in_code=0
    for match_str in code_match.findall(s):
        link_count_in_code+=len(link_match.findall(match_str))
    return len(link_match.findall(s))-link_count_in_code

In [15]:
import numpy as np

In [16]:
def extract_features_from_body(s):
    num_code_lines=0
    link_count_in_code=0
    code_free_s=s
    for match_str in code_match.findall(s):
        num_code_lines+=match_str.count('\n')
        code_free_s=code_match.sub('',code_free_s)
        link_count_in_code+=len(link_match.findall(match_str))
        links=link_match.findall(s)
        link_count=len(links)
        link_count-=link_count_in_code
        html_free_s=re.sub(" +"," ",tag_match.sub('',code_free_s)).replace('\n','')
        link_free_s=html_free_s
    

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
clf=LogisticRegression()

In [19]:
clf

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
from sklearn.metrics import precision_recall_curve

In [21]:
import numpy as np

In [22]:
x=np.arange(5)

In [23]:
x

array([0, 1, 2, 3, 4])

In [24]:
np.logical_or(x<1,x>3)

array([ True, False, False, False,  True], dtype=bool)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
from sklearn.naive_bayes import MultinomialNB

In [27]:
from sklearn.pipeline import Pipeline

In [28]:
def create_ngram_model():
    tfidf_ngrams=TfidfVectorizer(ngram_range=(1,3),analyzer='word',binary=False)
    clf=MultinomialNB()
    pipeline=Pipeline([('vect',tfidf_ngrams),('clf',clf)])
    return pipeline

In [29]:
from sklearn.metrics import precision_recall_curve,auc
from sklearn.cross_validation import ShuffleSplit

In [30]:
def train_model(clf_factory,X,Y):
    cv=ShuffleSplit(n=len(X),n_iter=10,test_size=0.3,indices=True,random_state=0)
    scores=[]
    pr_scores=[]
    for train,test in cv:
        X_train,y_train=X[train],Y[train]
        X_test,y_test=X[test],Y[test]
        clf=clf_factory()
        clf.fit(X_train,y_train)
        train_score=clf.score(X_train,y_train)
        test_score=clf.score(X_test,y_test)
        scores.append(test_score)
        proba=clf.predict_proba(X_test)
        precision,recall,pr_thresholds=precision_recall_curve(y_test,proba[:,1])
        pr_scores.append(auc(recall,precision))
        summary=(np.mean(scores),np.std(scores),np.mean(pr_scores),np.std(pr_scores))
        print '%.3f\t%.3f\t%.3f\t%.3f'%summary

In [31]:
from sklearn.grid_search import GridSearchCV

In [32]:
from sklearn.metrics import f1_score

In [33]:
def grid_search_model(clf_facotry,X,Y):
    cv=ShuffleSplit(n=len(X),n_iter=10,test_size=0.3,indices=True,random_state=0)
    param_grid=dict(vect_ngram_range=[(1,1),(1,2),(1,3)],vect_min_df=[1,2],vect_stop_words=[None,'english'],
                    vect_smooth_idf=[False,True],vect_use_idf=[False,True],vect_sublinear_tf=[False,True],
                    clf_alpha=[0,0.01,0.05,0.1,0.5,1],)
    grid_search=GridSearchCV(clf_facotry(),param_grid=parma_grid,cv=cv,score_func=f1_score,verbose=10)
    grid_search.fit(X,Y)
    return grid_search.best_estimator_

In [34]:
import nltk

In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
nltk.word_tokenize('This is a good book.')

In [ ]:
import csv,collections

In [ ]:
def load_sent_word_net():
    sent_scores=collections.defaultdict(list)
    with open(os.path.join(DATA_DIR,'SentiWordNet_3.0.0_20130122.txt'),'r') as csvfile:
        reader=csv.reader(csvfile,delimiter='\t',quotechar='"')
        for line in reader:
            if line[0].startswith('#'):
                continue
            if len(line)==1:
                continue
            Pos,ID,PosScore,NegScore,SynsetTerms,Gloss=line
            if len(POS)==0 or len(ID)==0:
                continue
            for term in SynsetTerms.split(' '):
                term=term.split('#')[0]
                term=term.replace('-',' ').replace('_',' ')
                key='%s/%s'%(POS,term.split('#')[0])
                sent_scores[key].append((float(PosScore),float(NegScore)))
    for key,value in sent_scores.iteritems():
        sent_scores[key]=np.mean(value,axis=0)
    return sent_scores

In [ ]:
sent_word_net=load_sent_word_net()

In [ ]:
class LinguisticVectorizer(BaseEstimator):
    def get_feature_names(self):
        return np.array(['sent_neut','sent_pos','sent_neg','nouns','adjectives','verbs','adverbs',
                        'allcaps','exclamation','question','hashtag','mentioning'])
    def fit(self,documents,y=none):
        return self
    def _get_sentiments(self,d):
        sent=tuple(d.split())
        tagged=nltk.pos_tag(sent)
        pos_vals=[]
        neg_vals=[]
        nouns=0.
        adjectives=0.
        verbs=0.
        adverbs=0.
        for w,t in tagged:
            p,n=0,0
            sent_pos_type=None
            if t.startswith('NN'):
                sent_pos_type='n'
                nouns+=1
            elif t.startswith('JJ'):
                sent_pos_type='a'
                adjectives+=1
            elif t.startswith('VB'):
                sent_pos_type='v'
                verbs+=1
            elif t.startswith('RB'):
                sent_pos_type='r'
                adverbs+=1
            if sent_pos_type is not None:
                sent_word='%s/%s'%(sent_pos_type,w)
                if sent_word in sent_word_net:
                    p,n=sent_word_net[sent_word]
            pos_vals.append(p)
            pos_vals.append(n)
        l=len(sent)
        avg_pos_val=np.mean(pos_vals)
        avg_neg_val=np.mean(neg_vals)
        return [1-avg_pos_val-avg_neg_val,avg_pos_val,avg_neg_val,nouns/1,adjectives/1,verbs/1,adverbs/1]
    def transform(self,documents):
        obj_val,pos_val,neg_val,nouns,adjectives,verbs,adverbs=np.array([self._get_sentiments(d) for d in documents]).T
        allcaps=[]
        exclamation=[]
        question=[]
        hashtag=[]
        mentioning=[]
        for d in documents:
            allcaps.append(np.sum([t.isupper() for t in d.split() if len(t)>2]))
            exclamation.append(d.count('!'))
            question.append(d.count('?'))
            hashtag.append(d.cpint('#'))
            mentioning.append(d.count('@'))
        result=np.array([obj_val,pos_val,neg_val,nouns,adjectives,verbs,adverbs,allcaps,exclamation,question,hashtag,mentioning]).T
        return result

In [ ]:
def create_union_model(params=None):
    def preprocessor(tweet):
        tweet=tweet.lower()
        for k in emo_repl_order:
            tweet=tweet.replace(k,emo_repl[k])
        for r,repl in re_repl.iteritems():
            tweet=re.sub(r,repl,tweet)
        return tweet.replace('-',' ').replace('_',' ')
    tfidf_ngrams=TfidfVectorizer(preprocessor=preprocessor,analyzer='word')
    ling_stats=LinguisticVectorizer()
    all_features=FeatureUnion([('ling',ling_stats),('tfidf',tfidf_ngrams)])
    clf=MultinomialNB()
    pipeline=Pipeline([('all',all_features),('clf',clf)])
    if params:
        pipeline.set_params(**params)
    return pipeline